<a href="https://colab.research.google.com/github/0rina/0rina/blob/main/aryna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
def process_video_fixed_frames(video_path, frame_size=(64, 64), duration=10, fps_target=2):


    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps_original = cap.get(cv2.CAP_PROP_FPS)
    #frame_interval = frame_count // num_frames
    #print ("Frame interval:", frame_interval)

    video_duration = frame_count / fps_original  # Czas trwania wideo w sekundach
    print(f"Wideo trwa {video_duration:.2f} sekund")
    total_frames_to_extract = int(min(duration, video_duration) * fps_target)  # Liczba ramek do wyciągnięcia
    frame_interval = int(fps_original // fps_target)  # Interwał ramek do ekstrakcji (w oryginalnych FPS)


    #print (video_path, " fps: ", fps, "frames:", frame_count)
    frames = []

    frame_number = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if frame_number % frame_interval == 0 and len(frames) < total_frames_to_extract:
                frame_resized = cv2.resize(frame, frame_size)
                frame_normalized = frame_resized / 255.0  # Normalizacja
                frames.append(frame_normalized)

            frame_number += 1

    finally:
        cap.release()

    # Dopasowanie do stałej liczby ramek
    if len(frames) < total_frames_to_extract:
        while len(frames) < total_frames_to_extract:
            frames.append(frames[-1])
    elif len(frames) > total_frames_to_extract:
        frames = frames[:total_frames_to_extract]

    return np.array(frames)

def process_videos_and_split(folder_path, label, frame_size=(64, 64),  duration=10, fps_target=2):

    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]

    # Check if video_files is empty
    if not video_files:
        print(f"No video files found in {folder_path}")
        return [], [], [], [] # Return empty lists if no video files are found

    test_video1 = random.choice(video_files)
    test_video2 = random.choice(video_files)

    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for video_file in video_files:
       video_path = os.path.join(folder_path, video_file)
       video_data = process_video_fixed_frames(video_path, frame_size, duration, fps_target)

       if video_file == test_video1 or video_file == test_video2:
            X_test.append(video_data)
            Y_test.append(label)
       else:
            X_train.append(video_data)
            Y_train.append(label)

    return X_train, Y_train, X_test, Y_test

def prepare_dataset(root_folder, frame_size=(64, 64), num_frames=20):

    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for word_folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, word_folder)
        if os.path.isdir(folder_path):
            label = word_folder
            x_train, y_train, x_test, y_test = process_videos_and_split(folder_path, label, frame_size, num_frames)

            X_train.extend(x_train)
            Y_train.extend(y_train)
            X_test.extend(x_test)
            Y_test.extend(y_test)

    # Konwersja na numpy arrays
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    print (X_train.shape)
    print (Y_train.shape)
    print (X_test.shape)
    print (Y_test.shape)


    np.save(output_base_folder + "X_train.npy", X_train)
    np.save(output_base_folder + "Y_train.npy", Y_train)
    np.save(output_base_folder + "X_test.npy", X_test)
    np.save(output_base_folder + "Y_test.npy", Y_test)

    print(f"Zapisano pliki: X_train.npy, Y_train.npy, X_test.npy, Y_test.npy")

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


v=np.arange(26)
frame_interval = len(v)//25
print (frame_interval)
frames =[i for i in v if i % frame_interval==0]
len(frames)

In [13]:
import os

def process_video_fixed_frames(video_path, frame_size=(64, 64), num_frames=20):


    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = frame_count // num_frames
    print ("Frame interval:", frame_interval)

    print (video_path, " fps: ", fps, "frames:", frame_count)
    frames = []

    number = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if number % frame_interval==0:
                frame_resized = cv2.resize(frame, frame_size)
                frame_normalized = frame_resized / 255.0
                frames.append(frame_normalized)
                number += 1

    finally:
        cap.release()

    # Dopasowanie do stałej liczby ramek
    if len(frames) < num_frames:
        while len(frames) < num_frames:
            frames.append(frames[-1])
    elif len(frames) > num_frames:
        frames = frames[:num_frames]

    return np.array(frames)

def process_videos_and_split(folder_path, label, frame_size=(64, 64), num_frames=20):

    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]

    # Check if video_files is empty
    if not video_files:
        print(f"No video files found in {folder_path}")
        return [], [], [], [] # Return empty lists if no video files are found

    test_video1 = random.choice(video_files)
    test_video2 = random.choice(video_files)

    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        video_data = process_video_fixed_frames(video_path, frame_size, num_frames)

        if video_file == test_video1 or video_file == test_video2:
            X_test.append(video_data)
            Y_test.append(label)
        else:
            X_train.append(video_data)
            Y_train.append(label)

    return X_train, Y_train, X_test, Y_test

def prepare_dataset(root_folder, frame_size=(64, 64), num_frames=20):

    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for word_folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, word_folder)
        if os.path.isdir(folder_path):
            label = word_folder
            x_train, y_train, x_test, y_test = process_videos_and_split(folder_path, label, frame_size, num_frames)

            X_train.extend(x_train)
            Y_train.extend(y_train)
            X_test.extend(x_test)
            Y_test.extend(y_test)

    # Konwersja na numpy arrays
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    print (X_train.shape)
    print (Y_train.shape)
    print (X_test.shape)
    print (Y_test.shape)


    np.save(output_base_folder + "X_train.npy", X_train)
    np.save(output_base_folder + "Y_train.npy", Y_train)
    np.save(output_base_folder + "X_test.npy", X_test)
    np.save(output_base_folder + "Y_test.npy", Y_test)

    print(f"Zapisano pliki: X_train.npy, Y_train.npy, X_test.npy, Y_test.npy")

#root_folder = r"C:\Users\Arina\Desktop\inzynierka\dataset\SLtest"
#output_base_folder = r"C:\Users\Arina\Desktop\inzynierka\dataset\outputVideo"

root_folder = '/content/drive/My Drive/praca_inzynierska/dataset/SLtest'
output_base_folder = '/content/drive/My Drive/praca_inzynierska/dataset/SLtest/outputVideo2'

prepare_dataset(root_folder)

Frame interval: 1
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/00583.mp4  fps:  29.97002997002997 frames: 38
Frame interval: 1
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/00585.mp4  fps:  29.97002997002997 frames: 38
Frame interval: 3
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/65006.mp4  fps:  23.976075485966692 frames: 62
Frame interval: 1
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/00586.mp4  fps:  29.97002997002997 frames: 39
Frame interval: 2
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/00584.mp4  fps:  29.97002997002997 frames: 42
Frame interval: 4
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/accent_1.mp4  fps:  29.97 frames: 96
Frame interval: 1
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/accent_5.mp4  fps:  29.97002997002997 frames: 38
Frame interval: 4
/content/drive/My Drive/praca_inzynierska/dataset/SLtest/accent/accent_3.mp4  fps:  29.9700

In [16]:
    X_train, Y_train, X_test, Y_test, unique_labels = load_data()

    #parametry modelu
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4])
    num_classes = len(unique_labels)

    model = build_model(input_shape, num_classes)

    history = train_model(model, X_train, Y_train, X_test, Y_test, batch_size=32, epochs=25)
    model = load_model("sign_language_model.h5")

    video_names = [f"video_{i}.mp4" for i in range(len(X_test))]

    test_model(model, X_test, Y_test, unique_labels, video_names)

num_classes:  10
Epoch 1/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 986ms/step - accuracy: 0.1966 - loss: 5.1866 - val_accuracy: 0.2105 - val_loss: 2.3288
Epoch 2/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.2705 - loss: 2.2522 - val_accuracy: 0.1053 - val_loss: 2.4033
Epoch 3/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.3547 - loss: 2.1102 - val_accuracy: 0.1579 - val_loss: 2.3087
Epoch 4/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.3934 - loss: 2.0018 - val_accuracy: 0.1579 - val_loss: 2.3626
Epoch 5/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.3979 - loss: 1.9032 - val_accuracy: 0.1579 - val_loss: 2.3201
Epoch 6/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.3977 - loss: 1.8315 - val_accuracy: 0.1579 - val_loss: 2.5453
Epoch 7/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.4301 - loss: 1.6717 - val_accuracy: 0.1579 - val_loss: 2.6074
Epoch 8/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.4744 - loss: 1.4662 - val_accuracy: 0.2632 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Video: video_0.mp4, Predicted: accent, True: accent
Video: video_1.mp4, Predicted: about, True: accent
Video: video_2.mp4, Predicted: accept, True: accept
Video: video_3.mp4, Predicted: accept, True: accept
Video: video_4.mp4, Predicted: about, True: a lot
Video: video_5.mp4, Predicted: about, True: a lot
Video: video_6.mp4, Predicted: a, True: a
Video: video_7.mp4, Predicted: a, True: a
Video: video_8.mp4, Predicted: about, True: above
Video: video_9.mp4, Predicted: accident, True: above
Video: video_10.mp4, Predicted: accident, True: abdomen
Video: video_11.mp4, Predicted: accident, True: abdomen
Video: video_12.mp4, Predicted: accent, True: about
Video: video_13.mp4, Predicted: about, True: about
Video: video_14.mp4, Predicted: accident, True: accident
Video: video_15.mp4, Predicted: accident, True: accident
Video: video_16.mp4, Predicted: accomplish, True: accomplish
Video: video_17.mp4, Predicted: a lot, True: able
Video: video_18.mp4, Predic